---
title: NLP
subtitle: "Natural Language Processing and Named Entity Recognition"
author:
  - name: Charles Pletcher
    affiliations: Tufts University
    orcid: 0000-0003-2734-5511
    email: charles.pletcher@tufts.edu
license:
  code: MIT
date: 2025-04-06
---

# Natural Language Processing

NLP is a large field — we'll only be able to scratch the surface today. We've already started working with elements of NLP, however: tokenization is a common first step in NLP, and it is also an NLP problem in itself.

As we've discussed, tokenization is not as simple as breaking on whitespace, nor even as simple as breaking on whitespace and punctuation. How should we handle hyphenated words, for example? What about "U.K." or "U.S."?

## Named Entity Recognition

Another subtask of NLP is Named Entity Recognition (NER). NER itself is composed of other sub-problems like named entity classification ("What kind of entity is this?") and named entity linking ("To what specific entity does this refer?")

Today, we'll be focusing on named entity classification of Book 1 of Pausanias' _Periegesis_. We'll be able to look up these entities in a data dump from the Pleiades project and feed them back into ArcGIS along with their coordinates and other relevant information.

## Loading the data

First, let's read in the transcription of Book 1 that we'll be using.

In [1]:
from pathlib import Path

paus_filename = Path("./txt/tlg0525.tlg001.theoi-eng.txt")

with open(paus_filename) as f:
    book_1 = f.read()

Simple enough. Let's just peek at the data to make sure it looks sane.

In [2]:
book_1[100:200]

'e Sunium promontory stands out from the Attic land. When you have rounded the promontory you see a h'

Seems pretty reasonable to me!

## Installing spaCy

[spaCy](https://spacy.io) is a Python library for NLP. Unlike the [NLTK](https://nltk.org), which prioritizes teaching and research, spaCy generally provides one way of performing a given task. For our purposes, spaCy's guided approach will be more than sufficient.

To get started, install spaCy like any other Python library:

In [3]:
%pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 48.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 52.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-war

But for spaCy to perform anything of use, we also need to download a pretrained _model_. Models are essentially large mappings of tokens (or subtokens) to long matrixes (lists of lists) of numbers. The larger the model, the more accurately it can represent a text in numerical terms — but also the more expensive it is to run.

We'll use the medium model today, as it hits the sweet spot for accuracy and usability.

In [4]:
%run -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 60.4 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


With the model downloaded, we can now run the text of `book_1` through spaCy's NER pipeline.

In [5]:
import spacy
from spacy import displacy

# Load the model that we downloaded.
# If this line fails, make sure that
# you have downloaded the model that's
# referenced here.
nlp = spacy.load("en_core_web_md")

# Analyze `book_1` — this might take a bit.
doc = nlp(book_1)

In [15]:
# We're running these lines in a separate cell so that we don't
# need to run the full analysis each time we inspect the results.

ents = [(e.text, e.label_) for e in doc.ents 
        if e.label_ not in ("CARDINAL","ORDINAL")]

for ent in ents:
    print(ent)

('PTOLEMY THE GREAT', 'WORK_OF_ART')
('Attalus', 'ORG')
('Ptolemy', 'PERSON')
('Egypt', 'GPE')
('Mysians', 'ORG')
('Macedonians', 'NORP')
('Ptolemy', 'PERSON')
('Philip', 'GPE')
('Amyntas', 'ORG')
('Lagus', 'PERSON')
('Lagus', 'PERSON')
('Philip', 'PERSON')
('Ptolemy', 'GPE')
('Asia', 'LOC')
('Alexander', 'PERSON')
('Oxydracae', 'FAC')
('Aridaeus', 'ORG')
('Philip', 'PERSON')
('Egypt', 'GPE')
('Cleomenes', 'PERSON')
('Alexander', 'PERSON')
('Perdiccas', 'GPE')
('Macedonians', 'NORP')
('Alexander to Aegae', 'ORG')
('Macedonian', 'NORP')
('Memphis', 'GPE')
('Perdiccas', 'GPE')
('Egypt', 'GPE')
('Perdiccas', 'PERSON')
('Aridaeus', 'ORG')
('Philip', 'PERSON')
('Alexander', 'PERSON')
('Roxana', 'PERSON')
('Oxyartes', 'PERSON')
('Alexander', 'PERSON')
('Ptolemy', 'GPE')
('Egypt', 'GPE')
('Egypt', 'GPE')
('Macedonians', 'NORP')
('Perdiccas', 'GPE')
('Ptolemy', 'PERSON')
('Syrians', 'NORP')
('Phoenicia', 'GPE')
('Seleucus', 'GPE')
('Antiochus', 'NORP')
('Antigonus', 'LOC')
('Antigonus', 'LOC')

In [7]:
doc.ents[0].start
# can also do for key words in context (word token, -4, +5)
# also read documentation/website

0

:::{note}
What is the type of the results in `ents`?
:::{answer}
e.text, e.label = string
otherwise "span objects"
touples of strings

In [11]:
import random

my_ents = random.sample(ents,20)

my_ents

[('Themistocles', 'ORG'),
 ('Megarus', 'PERSON'),
 ('Asia', 'LOC'),
 ('First', 'ORDINAL'),
 ('1.37.7', 'CARDINAL'),
 ('Macedonians', 'NORP'),
 ('Persians', 'NORP'),
 ('Oreithyia', 'PERSON'),
 ('Oceanus', 'GPE'),
 ('1.8.2', 'CARDINAL'),
 ('2.2.5', 'CARDINAL'),
 ('Acestium', 'PERSON'),
 ('Greece', 'GPE'),
 ('Neoptolemus', 'GPE'),
 ('Brauron', 'ORG'),
 ('Lacedaemonian', 'PERSON'),
 ('fourth', 'ORDINAL'),
 ('Leucippus', 'GPE'),
 ('Alexander', 'PERSON'),
 ('Aethiopian', 'NORP')]

## Looking up coordinates

While these results are far from perfect — "Hyllus," at least in my practice runs, was classified as a "PRODUCT" rather than a "PERSON" — they're fairly useful in broad strokes for our purposes.

But we still need to add coordinates, and we have over 4000 entities to link. How can we go about doing this scalably?

## Build a search tool

All of the data we need is available through [Pleiades](https://pleiades.stoa.org) and [ToposText](https://topostext.org), but the strings that are labeled by our NER model might not match the titles of places available from these sources. We could build a search index that lets us match titles mor flexibly, but that is beyond the scope of our work for today.

## Annotate by hand

Instead, working in groups, choose about **20** places from the NER list that you would like to map. You could even pull them out randomly, if you'd like.

Then, using Pleiades's own search tool, find the coordinates for each location. Store this data, along with any contextual information or descriptions that you deem relevant, in a CSV or spreadsheet that you can upload to ArcGIS.

:::{note}
Can you also include a `count` parameter for how often each place is mentioned in Book 1?
:::

If you find that your group is working particularly quickly, grab another 10 placenames, or experiment with mapping specific sections of Pausanias' text.

In [8]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")

doc = nlp("""

PTOLEMY THE GREAT (HISTORY)

[1.6.1] VI. But as to the history of Attalus and Ptolemy, it is more ancient in point of time, so that tradition no longer remains, and those who lived with these kings for the purpose of chronicling their deeds fell into neglect even before tradition failed. Where fore it occurred to me to narrate their deeds also, and how the sovereignty of Egypt, of the Mysians and of the neighboring peoples fell into the hands of their fathers.

[1.6.2] 27 The Macedonians consider Ptolemy to be the son of Philip, the son of Amyntas, though putatively the son of Lagus, asserting that his mother was with child when she was married to Lagus by Philip. And among the distinguished acts of Ptolemy in Asia they mention that it was he who, of Alexander's companions, was foremost in succoring him when in danger among the Oxydracae. After the death of Alexander,28 by withstanding those who would have conferred all his empire upon Aridaeus, the son of Philip, he became chiefly responsible for the division of the various nations into the kingdoms.

[1.6.3] He crossed over to Egypt in person, and killed Cleomenes, whom Alexander had appointed satrap of that country, considering him a friend of Perdiccas, and therefore not faithful to himself; and the Macedonians who had been entrusted with the task of carrying the corpse of Alexander to Aegae, he persuaded to hand it over to him. And he proceeded to bury it with Macedonian rites in Memphis, but, knowing that Perdiccas would make war, he kept Egypt garrisoned. And Perdiccas took Aridaeus, son of Philip, and the boy Alexander, whom Roxana, daughter of Oxyartes, had borne to Alexander, to lend color to the campaign, but really he was plotting to take from Ptolemy his kingdom in Egypt. But being expelled from Egypt, and having lost his reputation as a soldier, and being in other respects unpopular with the Macedonians, he was put to death by his body guard.

[1.6.4] The death of Perdiccas immediately raised Ptolemy to power, who both reduced the Syrians and Phoenicia, and also welcomed Seleucus, son of Antiochus, who was in exile, having been expelled by Antigonus; he further himself prepared to attack Antigonus. He prevailed on Cassander, son of Anti pater, and Lysimachus, who was king in Thrace, to join in the war, urging that Seleucus was in exile and that the growth of the power of Antigonus was dangerous to them all.

[1.6.5] For a time Antigonus pre pared for war, and was by no means confident of the issue; but on learning that the revolt of Cyrene had called Ptolemy to Libya, he immediately reduced the Syrians and Phoenicians by a sudden inroad, handed them over to Demetrius, his son, a man who for all his youth had already a reputation for good sense, and went down to the Hellespont. But he led his army back without crossing, on hearing that Demetrius had been overcome by Ptolemy in battle. But Demetrius had not altogether evacuated the country before Ptolemy, and having surprised a body of Egyptians, killed a few of them. Then on the arrival of Antigonus Ptolemy did not wait for him but returned to Egypt.

[1.6.6] When the winter was over, Demetrius sailed to Cyprus and overcame in a naval action Menelaus, the satrap of Ptolemy, and afterwards Ptolemy him self, who had crossed to bring help. Ptolemy fled to Egypt, where he was besieged by Antigonus on land and by Demetrius with a fleet. In spite of his extreme peril Ptolemy saved his empire by making a stand with an army at Pelusium while offering resistance with warships from the river. Antigonus now abandoned all hope of reducing Egypt in the circumstances, and dispatched Demetrius against the Rhodians with a fleet and a large army, hoping, if the island were won, to use it as a base against the Egyptians. But the Rhodians displayed daring and ingenuity in the face of the besiegers, while Ptolemy helped them with all the forces he could muster.

[1.6.7] Antigonus thus failed to reduce Egypt or, later, Rhodes, and shortly afterwards he offered battle to Lysimachus, and to Cassander and the army of Seleucus, lost most of his forces, and was himself killed, having suffered most by reason of the length of the war with Eumenes. Of the kings who put down Antigonus I hold that the most wicked was Cassander, who although he had recovered the throne of Macedonia with the aid of Antigonus, nevertheless came to fight against a benefactor.

[1.6.8] After the death of Antigonus, Ptolemy again reduced the Syrians and Cyprus, and also restored Pyrrhus to Thesprotia on the mainland. Cyrene rebelled; but Magas, the son of Berenice (who was at this time married to Ptolemy) captured Cyrene in the fifth year of the rebellion. If this Ptolemy really was the son of Philip, son of Amyntas, he must have inherited from his father his passion for women, for, while wedded to Eurydice, the daughter of Antipater, although he had children he took a fancy to Berenice, whom Antipater had sent to Egypt with Eurydice. He fell in love with this woman and had children by her, and when his end drew near he left the kingdom of Egypt to Ptolemy (from whom the Athenians name their tribe) being the son of Berenice and not of the daughter of Antipater.

[1.7.1] VII. This Ptolemy fell in love with Arsinoe, his full sister, and married her, violating herein Macedonian custom, but following that of his Egyptian subjects. Secondly he put to death his brother Argaeus, who was, it is said, plotting against him; and he it was who brought down from Memphis the corpse of Alexander. He put to death another brother also, son of Eurydice, on discovering that he was creating disaffection among the Cyprians. Then Magas, the half-brother of Ptolemy, who had been entrusted with the governorship of Cyrene by his mother Berenice--she had borne him to Philip, a Macedonians but of no note and of lowly origin--induced the people of Cyrene to revolt from Ptolemy and marched against Egypt.

[1.7.2] Ptolemy fortified the entrance into Egypt and awaited the attack of the Cyrenians. But while on the march Magas was in formed that the Marmaridae,a tribe of Libyan nomads, had revolted, and thereupon fell back upon Cyrene. Ptolemy resolved to pursue, but was checked owing to the following circumstance. When he was preparing to meet the attack of Magas, he engaged mercenaries, including some four thousand Gauls. Discovering that they were plotting to seize Egypt, he led them through the river to a deserted island. There they perished at one another's hands or by famine.

[1.7.3] Magas, who was married to Apame, daughter of Antiochus, son of Seleucus, persuaded Antiochus to break the treaty which his father Seleucus had made with Ptolemy and to attack Egypt. When Antiochus resolved to attack, Ptolemy dispatched forces against all the subjects of Antiochus, freebooters to overrun the lands of the weaker, and an army to hold back the stronger, so that Antiochus never had an opportunity of attacking Egypt. I have already stated how this Ptolemy sent a fleet to help the Athenians against Antigonus and the Macedonians, but it did very little to save Athens. His children were by Arsinoe, not his sister, but the daughter of Lysimachus. His sister who had wedded him happened to die before this, leaving no issue, and there is in Egypt a district called Arsinoites after her.

[1.8.1] VIII. It is pertinent to add here an account of Attalus, because he too is one of the Athenian eponymoi. A Macedonian of the name of Docimus, a general of Antigonus, who afterwards surrendered both himself and his property to Lysimachus, had a Paphlagonian eunuch called Philetaerus. All that Philetaerus did to further the revolt from Lysimachus, and how he won over Seleucus, will form an episode in my account of Lysimachus. Attalus, however, son of Attalus and nephew of Philetaerus, received the kingdom from his cousin Eumenes, who handed it over. The greatest of his achievements was his forcing the Gauls to retire from the sea into the country which they still hold.

[1.8.2] After the statues of the eponymoi come statues of gods, Amphiaraus, and Eirene (Peace) carrying the boy Plutus (Wealth). Here stands a bronze figure of Lycurgus,29 son of Lycophron, and of Callias, who, as most of the Athenians say, brought about the peace between the Greeks and Artaxerxes, son of Xerxes.30 Here also is Demosthenes, whom the Athenians forced to retire to Calauria, the island off Troezen, and then, after receiving him back, banished again after the disaster at Lamia.

[1.8.3] Exiled for the second time31 Demosthenes crossed once more to Calauria, and committed suicide there by taking poison, being the only Greek exile whom Archias failed to bring back to Antipater and the Macedonians. This Archias was a Thurian who undertook the abominable task of bringing to Antipater for punishment those who had opposed the Macedonians before the Greeks met with their defeat in Thessaly. Such was Demosthenes' reward for his great devotion to Athens. I heartily agree with the remark that no man who has unsparingly thrown himself into politics trusting in the loyalty of the democracy has ever met with a happy death.

[1.8.4] Near the statue of Demosthenes is a sanctuary of Ares, where are placed two images of Aphrodite, one of Ares made by Alcamenes, and one of Athena made by a Parian of the name of Locrus. There is also an image of Enyo, made by the sons of Praxiteles. About the temple stand images of Heracles, Theseus, Apollo binding his hair with a fillet, and statues of Calades,32 who it is said framed laws33 for the Athenians, and of Pindar, the statue being one of the rewards the Athenians gave him for praising them in an ode.

[1.8.5] Hard by stand statues of Harmodius and Aristogiton, who killed Hipparchus.34 The reason of this act and the method of its execution have been related by others; of the figures some were made by Critius,35 the old ones being the work of Antenor. When Xerxes took Athens after the Athenians had abandoned the city he took away these statues also among the spoils, but they were afterwards restored to the Athenians by Antiochus.

[1.8.6] Before the entrance of the theater which they call the Odeum (Music Hall) are statues of Egyptian kings. They are all alike called Ptolemy, but each has his own surname. For they call one Philometor, and another Philadelphus, while the son of Lagus is called Soter, a name given him by the Rhodians. Of these, Philadelphus is he whom I have mentioned before among the eponymoi, and near him is a statue of his sister Arsinoe.

          PTOLEMY PHILOMETOR OF EGYPT (HISTORY)

[1.9.1] IX. The one called Philometor is eighth in descent from Ptolemy son of Lagus, and his surname was given him in sarcastic mockery, for we know of none of the kings who was so hated by his mother. Although he was the eldest of her children she would not allow him to be called to the throne, but prevailed on his father before the call came to send him to Cyprus. Among the reasons assigned for Cleopatra's enmity towards her son is her expectation that Alexander the younger of her sons would prove more subservient, and this consideration induced her to urge the Egyptians to choose Alexander as king.

[1.9.2] When the people offered opposition, she dispatched Alexander for the second time to Cyprus, ostensibly as general, but really because she wished by his means to make Ptolemy more afraid of her. Finally she covered with wounds those eunuchs she thought best disposed, and presented them to the people, making out that she was the victim of Ptolemy's machinations, and that he had treated the eunuchs in such a fashion. The people of Alexandria rushed to kill Ptolemy, and when he escaped on board a ship, made Alexander, who returned from Cyprus, their king.

[1.9.3] Retribution for the exile of Ptolemy came upon Cleopatra, for she was put to death by Alexander, whom she herself had made to be king of the Egyptians. When the deed was discovered, and Alexander fled in fear of the citizens, Ptolemy returned and for the second time assumed control of Egypt. He made war against the Thebans, who had revolted, reduced them two years after the revolt, and treated them so cruelly that they were left not even a memorial of their former prosperity, which had so grown that they surpassed in wealth the richest of the Greeks, the sanctuary of Delphi and the Orchomenians. Shortly after this Ptolemy met with his appointed fate, and the Athenians, who had been benefited by him in many ways which I need not stop to relate, set up a bronze likeness of him and of Berenice, his only legitimate child.

LYSIMACHUS OF MACEDONIA (HISTORY)

[1.9.4] After the Egyptians come statues of Philip and of his son Alexander. The events of their lives were too important to form a mere digression in another story. Now the Egyptians had their honors bestowed upon them out of genuine respect and because they were benefactors, but it was rather the sycophancy of the people that gave them to Philip and Alexander, since they set up a statue to Lysimachus also not so much out of goodwill as because they thought to serve their immediate ends.

[1.9.5] This Lysimachus was a Macedonian by birth and one of Alexander's body-guards, whom Alexander once in anger shut up in a chamber with a lion, and afterwards found that he had overpowered the brute. Henceforth he always treated him with respect, and honored him as much as the noblest Macedonians. After the death of Alexander, Lysimachus ruled such of the Thracians, who are neighbors of the Macedonians, as had been under the sway of Alexander and before him of Philip. These would comprise but a small part of Thrace. If race be compared with race no nation of men except the Celts are more numerous than the Thracians taken all together, and for this reason no one before the Romans reduced the whole Thracian population. But the Romans have subdued all Thrace, and they also hold such Celtic territory as is worth possessing, but they have intentionally overlooked the parts that they consider useless through excessive cold or barrenness.

[1.9.6] Then Lysimachus made war against his neighbours, first the Odrysae, secondly the Getae and Dromichaetes. Engaging with men not unversed in warfare and far his superiors in number, he himself escaped from a position of extreme danger, but his son Agathocles, who was serving with him then for the first time, was taken prisoner by the Getae. Lysimachus met with other reverses afterwards, and attaching great importance to the capture of his son made peace with Dromicliaetes, yielding to the Getic king the parts of his empire beyond the Ister, and, chiefly under compulsion, giving him his daughter in marriage. Others say that not Agathocles but Lysimachus himself was taken prisoner, regaining his liberty when Agathocles treated with the Getic king on his behalf. On his return he married to Agathocles Lysandra, the daughter of Ptolemy, son of Lagus, and of Eurydice.

[1.9.7] He also crossed with a fleet to Asia and helped to overthrow the empire of Antigonus.36 He founded also the modern city of Ephesus as far as the coast, bringing to it as settlers people of Lebedos and Colophon, after destroying their cities, so that the iambic poet Phoenix com posed a lament for the capture of Colophon. Mermesianax, the elegiac writer, was, I think, no longer living, otherwise he too would certainly have been moved by the taking of Colophon to write a dirge. Lysimachus also went to war with Pyrrhus, son of Aeacides. Waiting for his departure from Epeirus (Pyrrhus was of a very roving disposition) he ravaged Epeirus until he reached the royal tombs.

[1.9.8] The next part of the story is incredible to me, but Hieronymus the Cardian37 relates that he destroyed the tombs and cast out the bones of the dead. But this Hieronymus has a reputation generally of being biased against all the kings except Antigonus, and of being unfairly partial towards him. As to the treatment of the Epeirot graves, it is perfectly plain that it was malice that made him record that a Macedonian desecrated the tombs of the dead. Besides, Lysimachus was surely aware that they were the ancestors not of Pyrrhus only but also of Alexander. In fact Alexander was an Epeirot and an Aeacid on his mother's side, and the subsequent alliance between Pyrrhus and Lysimachus proves that even as enemies they were not irreconcilable. Possibly Hieronymus had grievances against Lysimachus, especially his destroying the city of the Cardians and founding Lysimachea in its stead on the isthmus of the Thracian Chersonesus.

[1.10.1] X. As long as Aridaeus reigned, and after him Cassander and his sons, friendly relations continued between Lysimachus and Macedon. But when the kingdom devolved upon Demetrius, son of Antigonus, Lysimachus, henceforth expecting that war would be declared upon him by Demetrius, resolved to take aggressive action. He was aware that Demetrius inherited a tendency to aggrandise, and he also knew that he visited Macedonia at the summons of Alexander and Cassander, and on his arrival murdered Alexander himself38 and ruled the Macedonians in his stead.

[1.10.2] Therefore encountering Demetrius at Amphipolis he came near to being expelled from Thrace,39 but on Pyrrhus' coming to his aid he mastered Thrace and afterwards extended his empire at the expense of the Nestians and Macedonians. The greater part of Macedonia was under the control of Pyrrhus himself, who came from Epeirus with an army and was at that time on friendly terms with Lysimachus. When however Demetrius crossed over into Asia and made war on Seleucus, the alliance between Pyrrhus and Lysimachus lasted only as long as Demetrius continued hostilities; when Demetrius submitted to Seleucus, the friendship between Lysimachus and Pyrrhus was broken, and when war broke out Lysimachus fought against Antigonus son of Demetrius and against Pyrrhus himself, had much the better of the struggle, conquered Macedonia and forced Pyrrhus to retreat to Epeirus.

[1.10.3] Love is wont to bring many calamities upon men. Lysimachus, although by this time of mature age and considered happy in respect of his children, and although Agathocles had children by Lysandra, nevertheless married Lysandra's sister Arsinoe. This Arsinoe, fearing for her children, lest on the death of Lysimachus they should fall into the hands of Agathocles, is said for this reason to have plotted against Agathocles. Historians have already related how Arsinoe fell in love with Agathocles, and being unsuccessful they say that she plotted against his life. They say also that Lysimachus discovered later his wife's machinations, but was by this time powerless, having lost all his friends.

[1.10.4] Since Lysimachus, then, overlooked Arsinoe's murder of Agathocles, Lysandra fled to Seleucus, taking with her her children and her brothers, who were taking refuge with Ptolemy and finally adopted this course. They were accompanied on their flight to Seleucus by Alexander who was the son of Lysimachus by an Odrysian woman. So they going up to Babylon entreated Seleucus to make war on Lysimachus. And at the same time Philetaerus, to whom the property of Lysimachus had been entrusted, aggrieved at the death of Agathocles and suspicious of the treatment he would receive at the hands of Arsinoe, seized Pergamus on the Caicus, and sending a herald offered both the property and himself to Seleucus.

[1.10.5] Lysimachus hearing of all these things lost no time in crossing into Asia,40 and assuming the initiative met Seleucus, suffered a severe defeat and was killed. Alexander, his son by the Odrysian woman, after interceding long with Lysandra, won his body and afterwards carried it to the Chersonesus and buried it, where his grave is still to be seen between the village of Cardia and Pactye.

[1.11.1] XI. Such was the history of Lysimachus. The Athenians have also a statue of Pyrrhus. This Pyrrhus was not related to Alexander, except by ancestry. Pyrrhus was son of Aeacides, son of Arybbas, but Alexander was son of Olympias, daughter of Neoptolemus, and the father of Neoptolemus and Aryblas was Alcetas, son of Tharypus. And from Tharypus to Pyrrhus, son of Achilles, are fifteen generations. Now Pyrrhus was the first who after the capture of Troy disdained to return to Thessaly, but sailing to Epeirus dwelt there because of the oracles of Helenus. By Hermione Pyrrhus had no child, but by Andromache he had Molossus, Pielus, and Pergamus, who was the youngest. Helenus also had a son, Cestrinus, being married to Andromache after the murder of Pyrrhus at Delphi.

[1.11.2] Helenus on his death passed on the kingdom to Molossus, son of Pyrrhus, so that Cestrinus with volunteers from the Epeirots took possession of the region beyond the river Thyamis, while Pergamus crossed into Asia and killed Areius, despot in Teuthrania, who fought with him in single combat for his kingdom, and gave his name to the city which is still called after him. To Andromache, who accompanied him, there is still a shrine in the city. Pielus remained behind in Epeirus, and to him as ancestor Pyrrhus, the son of Aeacides, and his fathers traced their descent, and not to Molossus.

PYRRHUS OF MACEDONIA (HISTORY)

[1.11.3] Down to Alcetas, son of Tharypus, Epeirus too was under one king. But the sons of Alcetas after a quarrel agreed to rule with equal authority, remaining faithful to their compact; and afterwards, when Alexander, son of Neoptolemus, died among the Leucani, and Olympias returned to Epeirus through fear of Antipater, Aeacides, son of Arybbas, continued in allegiance to Olympias and joined in her campaign against Aridaeus and the Macedonians, although the Epeirots refused to accompany him.

[1.11.4] Olympias on her victory behaved wickedly in the matter of the death of Aridaeus, and much more wickedly to certain Macedonians, and for this reason was considered to have deserved her subsequent treatment at the hands of Cassander; so Aeacides at first was not received even by the Epeirots because of their hatred of Olympias, and when after wards they forgave him, his return to Epeirus was next opposed by Cassander. When a battle occurred at Oeneadae between Philip, brother of Cassander, and Aeacides, Aeacides was wounded and shortly after met his fate.41

[1.11.5] The Epeirots accepted Alcetas as their king, being the son of Arybbas and the elder brother of Aeacides, but of an uncontrollable temper and on this account banished by his father. Immediately on his arrival he began to vent his fury on the Epeirots, until they rose up and put him and his children to death at night. After killing him they brought back Pyrrhus, son of Aeacides. No sooner had he arrived than Cassander made war upon him, while he was young in years and before he had consolidated his empire. When the Macedonians attacked him, Pyrrhus went to Ptolemy, son of Lagus, in Egypt. Ptolemy gave him to wife the half-sister of his children, and restored him by an Egyptian force.

[1.11.6] The first Greeks that Pyrrhus attacked on becoming king were the Corcyraeans. He saw that the island lay off his own territory, and he did not wish others to have a base from which to attack him. My account of Lysimachus has already related how he fared, after taking Corcyra, in his war with Lysimachus, how he expelled Demetrius and ruled Macedonia until he was in turn expelled by Lysimachus, the most important of his achievements until he waged war against the Romans,

[1.11.7] being the first Greek we know of to do so. For no further battle, it is said, took place between Aeneas and Diomedes with his Argives. One of the many ambitions of the Athenians was to reduce all Italy, but the disaster at Syracuse42 prevented their trying conclusions with the Romans. Alexander, son of Neoptolemus, of the same family as Pyrrhus but older, died among the Leucani before he could meet the Romans in battle.

[1.12.1] XII. So Pyrrhus was the first to cross the Ionian Sea from Greece to attack the Romans.43 And even he crossed on the invitation of the Tarentines. For they were already involved in a war with the Romans, but were no match for them unaided. Pyrrhus was already in their debt, because they had sent a fleet to help him in his war with Corcyra, but the most cogent arguments of the Tarentine envoys were their accounts of Italy, how its prosperity was equal to that of the whole of Greece, and their plea that it was wicked to dismiss them when they had come as friends and suppliants in their hour of need. When the envoys urged these considerations, Pyrrhus remembered the capture of Troy, which he took to be an omen of his success in the war, as he was a descendant of Achilles making war upon a colony of Trojans.

[1.12.2] Pleased with this proposal, and being a man who never lost time when once he had made up his mind, he immediately proceeded to man war ships and to prepare transports to carry horses and men-at-arms. There are books written by men of no renown as historians, entitled “Memoirs.” When I read these I marvelled greatly both at the personal bravery of Pyrrhus in battle, and also at the forethought he displayed whenever a contest was imminent. So on this occasion also when crossing to Italy with a fleet he eluded the observation of the Romans, and for some time after his arrival they were unaware of his presence; it was only when the Romans made an attack upon the Tarentines that he appeared on the scene with his army, and his unexpected assault naturally threw his enemies into confusion.

[1.12.3] And being perfectly aware that he was no match for the Romans, he prepared to let loose against them his elephants. The first European to acquire elephants was Alexander, after subduing Porus and the power of the Indians; after his death others of the kings got them but Antigonus more than any; Pyrrhus captured his beasts in the battle with Demetrius. When on this occasion they came in sight the Romans were seized with panic, and did not believe they were animals.

[1.12.4] For although the use of ivory in arts and crafts all men obviously have known from of old, the actual beasts, before the Macedonians crossed into Asia, nobody had seen at all except the Indians themselves, the Libyans, and their neighbours. This is proved by Homer, who describes the couches and houses of the more prosperous kings as ornamented with ivory, but never mentions the beast; but if he had seen or heard about it he would, in my opinion have been much more likely to speak of it than of the battle between the Dwarf-men and cranes.44

[1.12.5] Pyrrhus was brought over to Sicily by an embassy of the Syracusans. The Carthaginians had crossed over and were destroying the Greek cities, and had sat down to invest Syracuse, the only one now remaining. When Pyrrhus heard this from the envoys he abandoned Tarentum and the Italiots on the coast, and crossing into Sicily forced the Carthaginians to raise the siege of Syracuse. In his self-conceit, although the Carthaginians, being Phoenicians of Tyre by ancient descent, were more experienced sea men than any other non-Greek people of that day, Pyrrhus was nevertheless encouraged to meet them in a naval battle, employing the Epeirots, the majority of whom, even after the capture of Troy, knew no thing of the sea nor even as yet how to use salt. Witness the words of Homer in the Odyssey:– Nothing they know of ocean, and mix not salt with their victuals. Hom. Od. 11.122

[1.13.1] XIII. Worsted on this occasion Pyrrhus put back with the remainder of his vessels to Tarentum. Here he met with a serious reverse, and his retirement, for he knew that the Romans would not let him depart without striking a blow, he contrived in the following manner. On his return from Sicily and his defeat, he first sent various dispatches to Asia and to Antigonus, asking some of the kings for troops, some for money, and Antigonus for both. When the envoys returned and their dispatches were delivered, he summoned those in authority, whether Epeirot or Tarentine, and without reading any of the dispatches declared that reinforcements would come. A report spread quickly even to the Romans that Macedonians and Asiatic tribes also were crossing to the aid of Pyrrhus. The Romans, on hearing this, made no move, but Pyrrhus on the approach of that very night crossed to the headlands of the mountains called Ceraunian.

[1.13.2] After the defeat in Italy Pyrrhus gave his forces a rest and then declared war on Antigonus, his chief ground of complaint being the failure to send reinforcements to Italy. Overpowering the native troops of Antigonus an his Gallic mercenaries he pursued them to the coast cities, and himself reduced upper Macedonia and the Thessalians. The extent of the fighting and the decisive character of the victory of Pyrrhus are shown best by the Celtic armour dedicated in the sanctuary of Itonian Athena between Pherae and Larisa, with this inscription on them:–

[1.13.3]

Pyrrhus the Molossian hung these shields
taken from the bold Gauls as a gift to Itonian
Athena, when he had destroyed all the host
of Antigonus. 'Tis no great marvel. The
Aeacidae are warriors now, even as they were of old.

These shields then are here, but the bucklers of the Macedonians themselves he dedicated to Dodonian Zeus. They too have an inscription:–

These once ravaged golden Asia, and brought
slavery upon the Greeks. Now ownerless
they lie by the pillars of the temple of Zeus,
spoils of boastful Macedonia.

Pyrrhus came very near to reducing Macedonia entirely, but,

[1.13.4] being usually readier to do what came first to hand, he was prevented by Cleonymus. This Cleonymus, who persuaded Pyrrhus to abandon his Macedonian adventure and to go to the Peloponnesus, was a Lacedaemonian who led an hostile army into the Lacedaemonian territory for a reason which I will relate after giving the descent of Cleonymus. Pausanias, who was in command of the Greeks at Plataea,45 was the father of Pleistoanax, he of Pausanias, and he of Cleombrotus, who was killed at Leuctra fighting against Epaminondas and the Thebans. Cleombrotus was the father of Agesipolis and Cleomenes, and, Agesipolis dying without issue, Cleomenes ascended the throne.

[1.13.5] Cleomenes had two sons, the elder being Acrotatus and the younger Cleonymus. Now Acrotatus died first; and when afterwards Cleomenes died, a claim to the throne was put forward by Areus son of Acrotatus, and Cleonymus took steps to induce Pyrrhus to enter the country. Before the battle of Leuctra46 the Lacedaemonians had suffered no disaster, so that they even refused to admit that they had yet been worsted in a land battle. For Leonidas, they said, had won the victory,47 but his followers were insufficient for the entire destruction of the Persians; the achievement of Demosthenes and the Athenians on the island of Sphacteria48 was no victory, but only a trick in war.

[1.13.6] Their first reverse took place in Boeotia, and they afterwards suffered a severe defeat at the hands of Antipater and the Macedonians.49 Thirdly the war with Demetrius50 came as an unexpected misfortune to their land. Invaded by Pyrrhus and seeing a hostile army for the fourth time, they arrayed themselves to meet it along with the Argives and Messenians who had come as their allies. Pyrrhus won the day, and came near to capturing Sparta without further fighting, but desisted for a while after ravaging the land and carrying off plunder.51 The citizens prepared for a siege, and Sparta even before this in the war with Demetrius had been fortified with deep trenches and strong stakes, and at the most vulnerable points with buildings as well.

[1.13.7] Just about this time, while the Laconian war was dragging on, Antigonus, having recovered the Macedonian cities, hastened to the Peloponnesus being well aware that if Pyrrhus were to reduce Lacedaemon and the greater part of the Peloponnesus, he would not return to Epeirus but to Macedonia to make war there again. When Antigonus was about to lead his army from Argos into Laconia, Pyrrhus himself reached Argos. Victorious once more he dashed into the city along with the fugitives, and his formation not unnaturally was broken up.

[1.13.8] When the fighting was now taking place by sanctuaries and houses, and in the narrow lanes, between detached bodies in different parts of the town, Pyrrhus left by himself was wounded in the head. It is said that his death52 was caused by a blow from a tile thrown by a woman. The Argives however declare that it was not a woman who killed him but Demeter in the likeness of a woman. This is what the Argives themselves relate about his end, and Lyceas, the guide for the neighborhood, has written a poem which confirms the story. They have a sanctuary of Demeter, built at the command of the oracle, on the spot where Pyrrhus died, and in it Pyrrhus is buried.

[1.13.9] I consider it remarkable that of those styled Aeacidae three met their end by similar heaven-sent means; if, as Homer says, Achilles was killed by Alexander, son of Priam, and by Apollo, if the Delphians were bidden by the Pythia to slay Pyrrhus, son of Achilles, and if the end of the son of Aeacides was such as the Argives say and Lyceas has described in his poem. The account, how ever, given by Hieronymus the Cardian is different, for a man who associates with royalty cannot help being a partial historian. If Philistus was justified in suppressing the most wicked deeds of Dionysius, because he expected his return to Syracuse, surely Hieronymus may be fully forgiven for writing to please Antigonus.

[1.14.1] XIV. So ended the period of Epeirot ascendancy. When you have entered the Odeum at Athens you meet, among other objects, a figure of Dionysus worth seeing. Hard by is a spring called Enneacrunos (Nine Jets), embellished as you see it by Peisistratus. There are cisterns all over the city, but this is the only fountain. Above the spring are two temples, one to Demeter and the Maid, while in that of Triptolemus is a statue of him. The accounts given of Triptolemus I shall write, omitting from the story as much as relates to Deiope.

[1.14.2] The Greeks who dispute most the Athenian claim to antiquity and the gifts they say they have received from the gods are the Argives, just as among those who are not Greeks the Egyptians compete with the Phrygians. It is said, then, that when Demeter came to Argos she was received by Pelasgus into his home, and that Chrysanthis, knowing about the rape of the Maid, related the story to her. Afterwards Trochilus, the priest of the mysteries, fled, they say, from Argos because of the enmity of Agenor, came to Attica and married a woman of Eleusis, by whom he had two children, Eubuleus and Triptolemus. That is the account given by the Argives. But the Athenians and those who with them . . . know that Triptolemus, son of Celeus, was the first to sow seed for cultivation.

[1.14.3] Some extant verses of Musaeus, if indeed they are to be included among his works, say that Triptolemus was the son of Oceanus and Earth; while those ascribed to Orpheus (though in my opinion the received authorship is again incorrect) say that Eubuleus and Triptolemus were sons of Dysaules, and that because they gave Demeter information about her daughter the sowing of seed was her reward to them. But Choerilus, an Athenian, who wrote a play called Alope, says that Cercyon and Triptolemus were brothers, that their mother was the daughter of Amphictyon, while the father of Triptolemus was Rarus, of Cercyon, Poseidon. After I had intended to go further into this story, and to describe the contents of the sanctuary at Athens, called the Eleusinium, I was stayed by a vision in a dream. I shall therefore turn to those things it is lawful to write of to all men.

[1.14.4] In front of this temple, where is also the statue of Triptolemus, is a bronze bull being led as it were to sacrifice, and there is a sitting figure of Epimenides of Cnossus,53 who they say entered a cave in the country and slept. And the sleep did not leave him before the fortieth year, and afterwards he wrote verses and purified Athens and other cities. But Thales who stayed the plague for the Lacedaemonians was not related to Epimenides in any way, and belonged to a different city. The latter was from Cnossus, but Thales was from Gortyn, according to Polymnastus of Colophon, who com posed a poem about him for the Lacedaemonians.

[1.14.5] Still farther of is a temple to Glory, this too being a thank-offering for the victory over the Persians, who had landed at Marathon. This is the victory of which I am of opinion the Athenians were proudest; while Aeschylus, who had won such renown for his poetry and for his share in the naval battles before Artemisium and at Salamis, recorded at the prospect of death nothing else, and merely wrote his name, his father's name, and the name of his city, and added that he had witnesses to his valor in the grove at Marathon and in the Persians who landed there.

[1.14.6] Above the Cerameicus and the portico called the King's Portico is a temple of Hephaestus. I was not surprised that by it stands a statue of Athena, be cause I knew the story about Erichthonius. But when I saw that the statue of Athena had blue eyes I found out that the legend about them is Libyan. For the Libyans have a saying that the Goddess is the daughter of Poseidon and Lake Tritonis, and for this reason has blue eyes like Poseidon.

[1.14.7] Hard by is a sanctuary of the Heavenly Aphrodite; the first men to establish her cult were the Assyrians, after the Assyrians the Paphians of Cyprus and the Phoenicians who live at Ascalon in Palestine; the Phoenicians taught her worship to the people of Cythera. Among the Athenians the cult was established by Aegeus, who thought that he was childless (he had, in fact, no children at the time) and that his sisters had suffered their misfortune because of the wrath of Heavenly Aphrodite. The statue still extant is of Parian marble and is the work of Pheidias. One of the Athenian parishes is that of the Athmoneis, who say that Porphyrion, an earlier king than Actaeus, founded their sanctuary of the Heavenly One. But the traditions current among the Parishes often differ altogether from those of the city.

[1.15.1] XV. As you go to the portico which they call painted, because of its pictures, there is a bronze statue of Hermes of the Market-place, and near it a gate. On it is a trophy erected by the Athenians, who in a cavalry action overcame Pleistarchus, to whose command his brother Cassander had entrusted his cavalry and mercenaries. This portico contains, first, the Athenians arrayed against the Lacedaemonians at Oenoe in the Argive territory.54 What is depicted is not the crisis of the battle nor when the action had advanced as far as the display of deeds of valor, but the beginning of the fight when the combatants were about to close.

[1.15.2] On the middle wall are the Athenians and Theseus fighting with the Amazons. So, it seems, only the women did not lose through their defeats their reckless courage in the face of danger; Themiscyra was taken by Heracles, and afterwards the army which they dispatched to Athens was destroyed, but nevertheless they came to Troy to fight all the Greeks as well as the Athenians them selves. After the Amazons come the Greeks when they have taken Troy, and the kings assembled on account of the outrage committed by Ajax against Cassandra. The picture includes Ajax himself, Cassandra and other captive women.

[1.15.3] At the end of the painting are those who fought at Marathon; the Boeotians of Plataea and the Attic contingent are coming to blows with the foreigners. In this place neither side has the better, but the center of the fighting shows the foreigners in flight and pushing one another into the morass, while at the end of the painting are the Phoenician ships, and the Greeks killing the foreigners who are scrambling into them. Here is also a portrait of the hero Marathon, after whom the plain is named, of Theseus represented as coming up from the under-world, of Athena and of Heracles. The Marathonians, according to their own account, were the first to regard Heracles as a god. Of the fighters the most conspicuous figures in the painting are Callimachus, who had been elected commander-in-chief by the Athenians, Miltiades, one of the generals, and a hero called Echetlus, of whom I shall make mention later.

[1.15.4] Here are dedicated brazen shields, and some have an inscription that they are taken from the Scioneans and their allies,55 while others, smeared with pitch lest they should be worn by age and rust, are said to be those of the Lacedaemonians who were taken prisoners in the island of Sphacteria.56

[1.16.1] XVI. Here are placed bronze statues, one, in front of the portico, of Solon, who composed the laws for the Athenians,57 and, a little farther away, one of Seleucus, whose future prosperity was foreshadowed by unmistakable signs. When he was about to set forth from Macedonia with Alexander, and was sacrificing at Pella to Zeus, the wood that lay on the altar advanced of its own accord to the image and caught fire without the application of a light.
""")

In [9]:
ents = [(e.text, e.label_) for e in doc.ents 
        if e.label_ in ("LOC","GPE")]

for ent in ents:
    print(ent)

('Egypt', 'GPE')
('Philip', 'GPE')
('Ptolemy', 'GPE')
('Asia', 'LOC')
('Egypt', 'GPE')
('Perdiccas', 'GPE')
('Memphis', 'GPE')
('Perdiccas', 'GPE')
('Egypt', 'GPE')
('Ptolemy', 'GPE')
('Egypt', 'GPE')
('Egypt', 'GPE')
('Perdiccas', 'GPE')
('Phoenicia', 'GPE')
('Seleucus', 'GPE')
('Antigonus', 'LOC')
('Antigonus', 'LOC')
('Thrace', 'GPE')
('Seleucus', 'GPE')
('Antigonus', 'LOC')
('Antigonus', 'LOC')
('Ptolemy', 'GPE')
('Libya', 'GPE')
('Hellespont', 'GPE')
('Antigonus Ptolemy', 'GPE')
('Egypt', 'GPE')
('Cyprus', 'GPE')
('Ptolemy', 'GPE')
('Egypt', 'GPE')
('Antigonus', 'LOC')
('Egypt', 'GPE')
('Egypt', 'GPE')
('Seleucus', 'GPE')
('Macedonia', 'GPE')
('Antigonus', 'LOC')
('Antigonus, Ptolemy', 'GPE')
('Cyprus', 'GPE')
('Magas', 'GPE')
('Ptolemy', 'GPE')
('Philip', 'GPE')
('Berenice', 'GPE')
('Egypt', 'GPE')
('Egypt', 'GPE')
('Ptolemy', 'GPE')
('Berenice', 'GPE')
('Memphis', 'GPE')
('Magas', 'GPE')
('Philip', 'GPE')
('Ptolemy', 'GPE')
('Egypt', 'GPE')
('Egypt', 'GPE')
('Magas', 'GPE')
('Ma

In [51]:
from collections import Counter

ents = [(e.text, e.label_) for e in doc.ents
        if e.label_ in ("LOC","GPE")]

Counter(ents)


Counter({('Egypt', 'GPE'): 19,
         ('Antigonus', 'LOC'): 17,
         ('Seleucus', 'GPE'): 12,
         ('Ptolemy', 'GPE'): 11,
         ('Macedonia', 'GPE'): 10,
         ('Asia', 'LOC'): 7,
         ('Athens', 'GPE'): 7,
         ('Argives', 'GPE'): 6,
         ('Cyprus', 'GPE'): 5,
         ('Magas', 'GPE'): 5,
         ('Philip', 'GPE'): 4,
         ('Neoptolemus', 'GPE'): 4,
         ('Italy', 'GPE'): 4,
         ('Perdiccas', 'GPE'): 3,
         ('Berenice', 'GPE'): 3,
         ('Agathocles', 'GPE'): 3,
         ('Poseidon', 'GPE'): 3,
         ('Memphis', 'GPE'): 2,
         ('Thrace', 'GPE'): 2,
         ('Philetaerus', 'GPE'): 2,
         ('Calauria', 'GPE'): 2,
         ('Thessaly', 'GPE'): 2,
         ('Cestrinus', 'LOC'): 2,
         ('Greece', 'GPE'): 2,
         ('Syracuse', 'GPE'): 2,
         ('Sparta', 'GPE'): 2,
         ('Phoenicia', 'GPE'): 1,
         ('Libya', 'GPE'): 1,
         ('Hellespont', 'GPE'): 1,
         ('Antigonus Ptolemy', 'GPE'): 1,
         ('A

In [53]:
len([(e.text, e.label_) for e in doc.ents
        if e.label_ in ("LOC","GPE")])

189

## Readings

- <https://doi.org/10.5334/johd.150>
- <https://doi.org/10.5281/zenodo.1193921>
- Kirsch, Adam. "Technology Is Taking Over English Departments." The New Republic, May 2, 2014
- @Blei2012
- @Brett2012
- @Mimno2012
- @Wellmon2015

## Homework

- Finish annotating placenames and uploading the results to an ArcGIS map
- Share a link to the map on Canvas